<a href="https://colab.research.google.com/github/GCandeas/Vend.IA/blob/main/Vend_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Vend.IA**

## Introdução

Este projeto visa exemplificar a implementação da inteligência artificial generativa Gemini para a precificação de produtos, proporcionando uma abordagem genérica e replicável para qualquer empreendimento com objetivos semelhantes.

A arquitetura adotada pelo projeto é composta por três etapas fundamentais: entrada de dados, processamento pelo modelo e saída de dados.

O propósito central é capacitar um modelo de IA Gemini para interpretar um prompt contendo informações específicas sobre um produto e, em resposta, gerar um valor numérico correspondente ao preço de venda desse produto.

Ao longo do processo, serão abordadas as seguintes etapas:

* Seleção dos dados de entrada;
* Criação de um contexto para os dados de entrada (engenharia de prompts);
* Aprimoramento do modelo (ajuste fino);
* Teste do modelo por meio de uma API.

Este projeto utilizará dados previamente preparados, permitindo focar na implementação do modelo. Tais dados são reais, originados de uma aplicação comercial em uso, proporcionando uma aproximação significativa com os desafios enfrentados no cotidiano.

Essa iniciativa visa resolver uma necessidade real de um cliente que enfrenta múltiplas negociações diárias. Trata-se de uma loja especializada na venda de peças automotivas, cuja rotina inclui negociações com diversas seguradoras visando garantir o sucesso das vendas.


## 1. Selecionando os Dados

O processo de construção de um modelo de precificação inicia-se com a identificação dos dados cruciais necessários para estabelecer o preço do produto em questão. Não apenas os dados em si são relevantes, mas também sua formatação, visando uma melhor compreensão por parte do modelo.

Considerando as necessidades do cliente, foi determinado que os dados seriam disponibilizados em formato JSON, incluindo os seguintes campos:

```
{
    "cod_produto": {
      "montadora": {
        "montadora": <nome_montadora>,
        "sigla_montadora": <sigla_sistema>
      },
      "id_conjunto": <id_conjuto_modelos>,
      "tipo": {
        "tipo": <descricao_tipo>,
        "sigla_tipo": <sigla_sistema>
      },
      "caracteristica": {
        "caracteristica": <descricao_caracteristica>,
        "id_caracteristica": <id_caracteristica_sistema>
      }
    },
    "fabricante": {
      "fabricante": <nome_fabricante>,
      "sigla_fabricante": <sigla_sistema>
    },
    "val_compra": <valor_compra>,
    "dat_compra": <data_compra>,
    "dat_venda": <data_venda>, -> dia atual
    "dias_estoque": <tempo_estocado>,
    "busca_web": [<descricao_tipo>,<descricao_caracteristica>,<modelo>,<ano_modelo>,<nome_fabricante>] -> palavras-chave para busca na internet
  }
```

A escolha dos dados deve refletir as informações utilizadas pelo usuário para precificar o produto, considerando também o processo realizado pelo usuário para chegar a um resultado, neste caso, o preço de venda.

## 2. Criando o Prompt

A construção de um prompt para este modelo requer experimentações e testes detalhados para alcançar um resultado satisfatório.

Neste projeto, adotaremos uma abordagem em que dividimos o input em duas partes distintas. Na primeira parte, forneceremos um contexto para o input. Esse contexto consiste em instruções claras sobre o que desejamos que o modelo faça, refletindo o processo realizado pelo usuário para atingir o resultado desejado.

Por outro lado, a segunda parte do input será composta pelos dados fornecidos em formato JSON. Estes dados são cruciais para alcançar o resultado desejado, neste caso, a precificação do produto.

O prompt pode ser exemplificado da seguinte forma:

```
Siga os passos a seguir. O campo "busca_web" do JSON contém um array de palavras-chave referentes a um produto específico. Faça uma pesquisa utilizando o Google Search com as palavras-chave fornecidas pelo campo "busca_web" e traga os 5 valores de venda mais relevantes encontrados. Considere apenas valores de produtos novos. O JSON fornecido contém informações relevantes sobre um produto. A partir dessas informações, extrapole um valor de venda para o produto em questão. Leve em consideração os valores encontrados pela pesquisa realizada no Google Search para extrapolar o valor de venda.

{
    "cod_produto": {
      "montadora": {
        "montadora": "fiat",
        "sigla_montadora": "FT"
      },
      "id_conjunto": 104,
      "tipo": {
        "tipo": "parabrisa",
        "sigla_tipo": "PB"
      },
      "caracteristica": {
        "caracteristica": "degrade",
        "id_caracteristica": 101
      }
    },
    "fabricante": {
      "fabricante": "pilkington",
      "sigla_fabricante": "PK"
    },
    "val_compra": 258,
    "dat_compra": "2022-06-10T13:30:01.142508",
    "dat_venda": "2022-10-06T13:09:05.166543",
    "dias_estoque": 118,
    "busca_web": ["parabrisa","degrade","fiorino bau",2015,"pilkington"]
  }
```

É importante ressaltar que o contexto permanecerá constante para todos os inputs, enquanto os dados (JSON) são variáveis e devem ser substituídos conforme cada input.

## 3. Aprimorando o Modelo (Tuning)

Para garantir resultados satisfatórios nesta situação, é fundamental ir além do prompt ideal. É necessário customizar o modelo, treiná-lo para processar com clareza o input e retornar o dado esperado.

Por isso, o tuning do modelo é essencial. A plataforma do Google AI Studio oferece de maneira intuitiva um método para isso.

Na plataforma, na barra de ferramentas à esquerda, encontra-se o botão "new tuned model", que nos direciona para uma tela onde podemos treinar o modelo com base em dados prévios no formato CSV.

O CSV consiste em uma série de inputs e outputs esperados, treinando a IA para que, ao receber determinado input, retorne o output esperado. Utilizei o documento "TRAINIG_DATA.csv" para fazer o tuning do modelo. Este documento possui 430 linhas, onde cada uma corresponde a uma entrada e sua saída esperada, seguindo a seguinte estrutura:

> *contexto, JSON, output*

Onde os campos "contexto" e "JSON" correspondem ao input do prompt, conforme definimos anteriormente.

Além disso, é necessário definir em "Advanced Settings" a quantidade de épocas, a taxa de aprendizagem e o batch size para treinar nosso modelo com nossos dados.

Para este projeto, realizei 3 treinamentos com configurações diferentes. No primeiro modelo, utilizei as configurações automáticas e observei que após 5 épocas, a taxa de erro ainda estava elevada. Então, treinei outro modelo com 10 épocas e percebi que a taxa de erros estava convergindo para 0. Porém, insatisfeito, fiz mais um treinamento com 15 épocas e observei a mesma convergência, porém desta vez já estava satisfatório o resultado. Decidi que este seria o modelo a ser utilizado neste projeto.

É importante ressaltar que é possível otimizar ainda mais o modelo, o que não foi feito devido a restrições de tempo para garantir a entrega dentro do prazo do desafio proposto.

Toda a documentação necessária está disponível em: [Tuning Guild](https://ai.google.dev/gemini-api/docs/model-tuning?hl=pt-br)

## 4. Testando os resultados com a API

Chegou o momento de testarmos nosso modelo usando a API do Google para enviar os prompts ao nosso modelo treinado e receber as respostas automaticamente, simulando sua utilização no dia a dia pelo cliente.

É importante ressaltar que estamos lidando com a parte do backend, e que praticamente todo esse processo é transparente para o cliente.

A seguir, apresento o trecho de código comentado e dividido em etapas para facilitar sua compreensão. Para a execução do teste, utilizamos o arquivo "EXT_TESTE_DADOS.json", que contém 50 registros não utilizados durante o tuning, proporcionando uma simulação real do dia a dia.

In [ ]:
# Executando o comando pip para instalar o pacote 'pip install google-generativeai'
!pip install -q -U google-generativeai

Obs.: Para termos acesso aos modelos 'tunados' é necessário uma autenticação especial, conforme explicado na documentação: [OAuth](https://ai.google.dev/palm_docs/oauth_quickstart?hl=pt-br)

In [ ]:
# Executando o comando que realiza autenticacao no gcloud
# Lembrando que para cada projeto sera nescessario um "client-id-file"
!gcloud auth application-default login \
    --no-browser \
    --client-id-file=/content/client_secret_371389574539-kn06ke8tuuqc0sdcl6mpoq39her7tk93.apps.googleusercontent.com.json \
    --scopes='https://www.googleapis.com/auth/cloud-platform,https://www.googleapis.com/auth/generative-language.tuning'

You are authorizing client libraries without access to a web browser. Please run the following command on a machine with a web browser and copy its output back here. Make sure the installed gcloud version is 372.0.0 or newer.

gcloud auth application-default login --remote-bootstrap="https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=371389574539-kn06ke8tuuqc0sdcl6mpoq39her7tk93.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgenerative-language.tuning&state=hZcEZPpm8A33J1VF4PHp0cO0hy1usc&access_type=offline&code_challenge=xB3VeexANud8dne_gp_yRWHWiEOmzgt9oIA7_lzS_Zc&code_challenge_method=S256&token_usage=remote"


Enter the output of the above command: https://localhost:8085/?state=hZcEZPpm8A33J1VF4PHp0cO0hy1usc&code=4/0AdLIrYd0ubmWxkYkyUsJ3shbGIuafEHZnjkrB4uhA_5RbFntOLM7SJUq3ixi33oJW7yNmQ&scope=https://www.googleapis.com/auth/cloud-platform%20https://www.googleapis.com/auth/generati

In [ ]:
# Importando a biblioteca da API do Google Gemini
import google.generativeai as genai

# Importando a biblioteca do Google colab para utilisacao do cofre de senhas (secrets)
from google.colab import userdata

# Importando a biblioteca de manipulacao de JSON do Python
import json

In [ ]:
# Definindo as configuracoes para a elaboracao das respostas
generation_config = {
  'temperature': 0.9,
  'candidate_count':1
}

# Definindo as configuracoes de "seguraca" para as respostas
safety_settings = [
]

In [ ]:
# Criando uma nova instancia do modelo generativo de acordo com as configuracoes
# Perceba que foi informado um nome customisado para o modelo
model = genai.GenerativeModel(
    model_name="tunedModels/vendia-tuned-e15b5-5jo772t1rh4w",
    generation_config=generation_config,
    safety_settings=safety_settings
)

In [ ]:
# Definindo varivel para armazenar o input que dara contexto ao prompt
prompt_context = 'Siga os passos a seguir. O campo "busca_web" do JSON contem um array de palavras chaves referentes a um produto especifico. Faça uma pesquisa utilizando o Google Search com as palavras chaves fornecidas pelo campo "busca_web" e traga os 5 valores de venda mais relevantes encontrados. Traga somente valores de produtos novos. O JSON fornecido contem informações relevantes sobre um produto. A partir destas informações extrapole um valor de venda para o referido produto. Leve em consideração os valores encontrados pela pesquisa realizada no  Google Search para estrapolar o valor de venda.'

In [ ]:
# Abrindo e carregando os dados do arquivo de dados para teste
# Os dados carregados sao salvos na variavel "data_test"
with open('EXT_TESTE_DADOS.json', mode='r') as f:
  data_test = json.load(f)

In [ ]:
# Definindo lista para armazenar a porcentagem de acerto de cada interacao
resultados = []

# Percorrendo os dados de teste e executando a API
for data in data_test:

  # Formatando o JSON para string
  # Salva o JSON formatado na variavel "prompt_json"
  prompt_json = json.dumps(data['input'], indent=4)

  # Definindo o prompt que sera enviado para API
  prompt_parts = [
      prompt_context,
      prompt_json
  ]

  # Envia o prompt para ser processado e armazena a respota obtida na variavel "model_response"
  model_response = model.generate_content(prompt_parts)

  # Extrai o valor obtido pelo modelo e o convert para inteiro
  response = int(model_response.text)

  # Definindo variavel para armazenar o valor esperado de acordo com os dados de teste
  esperado = int(data["output"])

  # Calcula a o valor absoluto entre a resposta e o valor esperado
  diferenca = abs(response - esperado)

  # Calcula o a porcentagem de acerto da resposta do modelo comparado com a saida esperada
  acerto = (1 - (diferenca / esperado)) * 100

  # Armazenando o resultado na lista
  resultados.append(acerto)

  # Saida para DEBUG: Exibe os calculos um a um
  #print(f' resposta: {response}\n esperado: {esperado}\n acerto: {acerto}%\n ')

# Calculando a media dos resultados
media = sum(resultados) / float(len(resultados))

# Imprimindo a media dos resultados
print(f"Total de input's processados: {len(resultados)}\nTaxa de acerto medio: {media}%")

Total de input's processados: 50
Taxa de acerto medio: 88.10572920896988%


## 5. Considerações finais
A conclusão dos testes demonstra um progresso animador em nossa jornada rumo à excelência. No entanto, reconhecemos que ainda há um vasto território a ser desbravado. Para alcançar patamares ainda mais altos, torna-se crucial mergulharmos nas nuances do comportamento de compra dos nossos clientes.

Ao desvendarmos as preferências e necessidades individuais de cada cliente, abrimos as portas para um novo patamar de vendas: não apenas satisfatórias, mas excepcionais. Através dessa compreensão profunda, podemos personalizar cada interação, transformando cada transação em uma experiência memorável e encantadora.